In [1]:
from PIL import Image 
import glob as gb
import numpy as np
import keras


Using TensorFlow backend.


In [2]:
image_width  = 16
image_height = 16

In [3]:
def load_hoda_image(path):
    all_files = gb.glob( path + "/*.bmp")
    data = []
    for i in range(len(all_files)):
        image_file = Image.open(all_files[0]) # open colour image
        image_file = image_file.convert('L')
        image_file = image_file.resize((image_width, image_height), Image.ANTIALIAS)
        np_array_file = np.array(image_file)
        data.append(np_array_file)
    return np.array(data)

In [4]:
def load_hoda_labels(path):
    all_files = gb.glob( path + "/*.bmp")
    data = []
    for i in range(len(all_files)):
        x = all_files[i].split("_")
        y = x[1].replace(".bmp" , "")
        data.append(y)
    return np.array(data)

In [5]:
def load_hoda():
    test_images = load_hoda_image('datasets/hoda/test')
    test_labels = load_hoda_labels('datasets/hoda/test')
    train_images = load_hoda_image('datasets/hoda/train')
    train_labels = load_hoda_labels('datasets/hoda/train')
    return train_images, train_labels, test_images, test_labels
    

In [6]:
train_images, train_labels, test_images, test_labels = load_hoda()

In [7]:
train_images.shape

(60000, 16, 16)

In [8]:
len(train_labels)

60000

In [9]:
train_labels

array(['4', '9', '9', ..., '2', '4', '0'], dtype='<U1')

In [10]:
test_images.shape

(20000, 16, 16)

In [11]:
len(test_labels)

20000

In [12]:
test_labels

array(['3', '9', '0', ..., '4', '9', '0'], dtype='<U1')

In [13]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(image_width * image_height,)))
network.add(layers.Dense(10, activation='softmax'))

In [14]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [15]:
train_images = train_images.reshape((60000, image_width * image_height))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((20000, image_width * image_height))
test_images = test_images.astype('float32') / 255

In [16]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [17]:
network.fit(train_images, train_labels, epochs=10, batch_size=128)

Epoch 1/10
60000/60000 [==============================] - 2s 33us/step - loss: 2.3138 - acc: 0.0995
Epoch 2/10
60000/60000 [==============================] - 2s 34us/step - loss: 2.3027 - acc: 0.0974
Epoch 3/10
60000/60000 [==============================] - 2s 35us/step - loss: 2.3027 - acc: 0.0992
Epoch 4/10
60000/60000 [==============================] - 2s 33us/step - loss: 2.3027 - acc: 0.0990
Epoch 5/10
60000/60000 [==============================] - 2s 34us/step - loss: 2.3027 - acc: 0.0978
Epoch 6/10
60000/60000 [==============================] - 2s 32us/step - loss: 2.3027 - acc: 0.0976
Epoch 7/10
60000/60000 [==============================] - 2s 33us/step - loss: 2.3027 - acc: 0.0972
Epoch 8/10
60000/60000 [==============================] - 2s 32us/step - loss: 2.3027 - acc: 0.0990
Epoch 9/10
60000/60000 [==============================] - 2s 31us/step - loss: 2.3027 - acc: 0.0983
Epoch 10/10
60000/60000 [==============================] - 2s 32us/step - loss: 2.3027 - acc: 0.0971

In [18]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

20000/20000 [==============================] - 1s 28us/step


In [19]:
print('test_acc:', test_acc)

test_acc: 0.1


In [20]:
def one_hot_encode_to_number(one_hot_coded):
    val = 0
    for i in range(0,10,1):
        val += i * one_hot_coded[i]
    return val

In [21]:
def one_hot_encode_percent_number(one_hot_coded):
    result = [];
    for i in range(0,10,1):
        per = one_hot_coded[i] * 100
        result.append(round(per,3))
    return result

In [22]:
def predicate_result(network, index):
    inp = np.array([test_images[index]])
    real_label = one_hot_encode_to_number(test_labels[index])
    result = one_hot_encode_percent_number(network.predict(inp)[0])
    return real_label, result

In [23]:
real_label, percent_result = predicate_result(network, 120)
print('real label is ', real_label)
for i in range(len(percent_result)):
    print(i, ':' , '{:6}%'.format(percent_result[i]))


real label is  0.0
0 :  7.977%
1 :  6.811%
2 : 13.878%
3 : 12.111%
4 :  9.179%
5 :  9.263%
6 : 10.117%
7 : 10.847%
8 :  9.111%
9 : 10.707%
